In [1]:
import sys
from tqdm import tqdm
sys.path.insert(0, '../')

import torch
from torch import nn
from torch.nn import functional as F

from attention import MultiHeadAttention
from encoder import Encoder

In [2]:
D_MODEL = 6
NUM_HEADS = 2
MAX_LEN = 4
encoder = Encoder(d_model=6, num_heads=2, max_len=4)

In [3]:
rands = torch.rand(10, 4, 6)

In [12]:
warning = torch.tensor([[[[-0.0005]],

         [[-0.1330]]],


        [[[-0.0731]],

         [[-0.0796]]],


        [[[-0.0494]],

         [[-0.1368]]],


        [[[-0.0534]],

         [[-0.1086]]]])

In [13]:
warning.size()

torch.Size([4, 2, 1, 1])

In [7]:
F.softmax(warning)

ipykernel_launcher:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[[[0.5331]],

         [[0.4669]]],


        [[[0.5016]],

         [[0.4984]]],


        [[[0.5218]],

         [[0.4782]]],


        [[[0.5138]],

         [[0.4862]]]])

In [10]:
torch.matmul(value, value.transpose(-1, -2)).size()

torch.Size([4, 2, 1, 1])

In [9]:
attn_raw.size(), value.size()

(torch.Size([4, 2, 1, 1]), torch.Size([4, 2, 1, 3]))

In [5]:
encoder(rands).size()

Value: tensor([[[[ 0.0982, -0.3208,  0.1822],
          [-0.1967, -0.1831,  0.3248],
          [-0.5634, -0.3581,  0.2186],
          [-0.2450, -0.1362,  0.0016]],

         [[ 0.2875, -0.3545, -0.1058],
          [ 0.1571, -0.0165, -0.2656],
          [ 0.4136,  0.1086, -0.5090],
          [ 0.1277, -0.1174, -0.1879]]],


        [[[ 0.0548, -0.1286,  0.2920],
          [-0.2804, -0.4597,  0.3159],
          [-0.3840, -0.2573,  0.1385],
          [-0.4327, -0.2655,  0.1239]],

         [[ 0.0772, -0.0358,  0.1251],
          [ 0.4670, -0.0497, -0.2630],
          [ 0.2421,  0.0061, -0.2739],
          [ 0.2861,  0.0898, -0.2361]]],


        [[[-0.1011, -0.1369,  0.3023],
          [-0.2612, -0.1382,  0.1947],
          [-0.3280, -0.5734,  0.0681],
          [-0.5033, -0.2594,  0.4877]],

         [[ 0.1291, -0.0219, -0.0682],
          [ 0.1201, -0.0549, -0.3217],
          [ 0.3903, -0.2089, -0.0475],
          [ 0.3944,  0.1730, -0.6897]]],


        [[[ 0.1319, -0.2355,  0.0248],


torch.Size([10, 4, 6])

In [27]:
torch.masked_fill(mask=mask, value=0)

TypeError: masked_fill() received an invalid combination of arguments - got (mask=Tensor, value=int, ), but expected one of:
 * (Tensor input, Tensor mask, Tensor value)
 * (Tensor input, Tensor mask, Number value)


In [25]:
mask = torch.tril(torch.ones(4,4), diagonal=0).bool()

In [12]:
pad_id = 0
vocab_size = 100
max_len = 10
hidden_dim = 6

def padding(data: list, pad_id: int=0) -> (list, int):
    max_len = len(max(data, key=len))
    output = [sample + [pad_id]*(max_len-len(sample)) for sample in tqdm(data)]
    return output, max_len

data = [
     [62, 13, 47, 39, 78, 33, 56, 13, 39, 29],
     [60, 96, 51, 32, 90, 44, 86, 71, 36, 18],
     [35, 45, 48, 65, 91, 99, 92, 10, 31, 21],
     [75, 51, 45, 48, 65, 91, 99, 11, 13, 28],
     [66, 88, 98, 47, 48, 65, 17, 13, 67, 99],
     ]

# data, max_len = padding(data)
data = torch.LongTensor(data)
print(data.shape)

torch.Size([5, 10])


In [13]:
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_dim)
embedding.weight.data.uniform_(-1, 1)
print('Weight initialized')

Weight initialized


In [14]:
X_embedded = embedding(data)
X_embedded.size()

torch.Size([5, 10, 6])

In [15]:
w_query = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Query
w_key = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Key
w_value = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Value

query = w_query(X_embedded)
key = w_key(X_embedded)
value = w_value(X_embedded)

In [19]:
attention = MultiHeadAttention(hidden_dim=hidden_dim, num_heads=2)
attention_mat = attention(query, key, value)

In [21]:
attention_mat.size()

torch.Size([5, 10, 6])

In [26]:
attn_layer_norm = nn.LayerNorm(normalized_shape=(10, 6))

In [25]:
attention_mat.size()[1:]

torch.Size([10, 6])

In [27]:
attn_layer_norm(attention_mat).size()

torch.Size([5, 10, 6])